<a href="https://colab.research.google.com/github/blankazselyke/Onlab/blob/DPO_2/DPO_LLama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip3 install transformers
!pip3 install datasets
!pip3 install trl
!pip install -q -U bitsandbytes
!pip install accelerate
!pip install peft
!pip install huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 26.5 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.0/102.0 kB 14.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-ma

In [3]:
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer,TrainingArguments, AutoModelForQuestionAnswering
from trl import DPOTrainer, SFTTrainer
from transformers import BitsAndBytesConfig
from torch import bfloat16
import torch
from peft import LoraConfig,PeftModel, PeftConfig, get_peft_model, AutoPeftModelForCausalLM
import warnings
import os
warnings.filterwarnings("ignore")

In [4]:
from huggingface_hub import notebook_login
notebook_login()

In [6]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # 4-bit quantization
    bnb_4bit_quant_type='nf4',  # Normalized float 4
    bnb_4bit_use_double_quant=True,  # Second quantization after the first
    bnb_4bit_compute_dtype=bfloat16  # Computation type
)
#Even though the weights are loaded in a 4-bit format,
#computations during forward and backward passes will be performed using bfloat16 data type,
#which is a 16-bit floating-point representation.
#bfloat16 strikes a balance between memory efficiency and computational precision.
model_name ="meta-llama/Meta-Llama-3-8B"

base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
)
base_model.config.use_cache = False

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/177 [00:00<?, ?B/s]

In [7]:
base_model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): Ll

In [8]:
def formatting_prompts_func(example):
  output_texts = []
  assert len(example["text"]) == len(example["label"])
  for i in range(len(example['text'])):
    text = f"### Input: ```{example['text'][i]}```\n ### Output: {example['label'][i]}"
    output_texts.append(text)
  return output_texts

In [9]:
import os
os.environ['TRANSFORMERS_NO_ADVISORY_WARNINGS'] = 'true'

In [10]:
sft_dataset = load_dataset("gbharti/finance-alpaca", split="train")
sft_datset_new = sft_dataset.select(range(10000))
sft_splitted_data = sft_datset_new.train_test_split(test_size=0.3,seed=42, shuffle=True)

sft_splitted_data["train"] = sft_splitted_data["train"].remove_columns(["input", "text"])
sft_splitted_data["test"] = sft_splitted_data["test"].remove_columns(["input", "text"])

sft_splitted_data["train"] = sft_splitted_data["train"].rename_column("output", "label")
sft_splitted_data["train"] = sft_splitted_data["train"].rename_column("instruction", "text")

sft_splitted_data["test"]  = sft_splitted_data["test"].rename_column("output", "label")
sft_splitted_data["test"]  = sft_splitted_data["test"].rename_column("instruction", "text")

Generating train split:   0%|          | 0/68912 [00:00<?, ? examples/s]

In [11]:
sft_splitted_data

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 7000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 3000
    })
})

In [12]:
tokenizer = AutoTokenizer.from_pretrained(model_name,trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
# training the supervised model
"""
 max_step = epoch*(total_example/batch_size)
 logging_step = total_example/batch_size
 loggers = logging_step/epoch
"""
output_path = "/content/drive/MyDrive/fine_tuning_llm_with_dop_peft3/models/"
config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["k_proj","v_proj","q_proj","out_proj"],
    bias="none",
    task_type="CAUSAL_LM",
)
MAX_STEPS =int(2 * len(sft_splitted_data["train"]) / 4)
args = TrainingArguments(
        output_path,
        overwrite_output_dir=True, # This reduces the amt of disk space that gets used.
        fp16=True,  # fp16 training to allow larger batch sizes to be used
        evaluation_strategy = "steps",
        save_strategy = "steps",
        learning_rate=5e-5,
        warmup_steps=500,
        warmup_ratio=0.1,
        do_train=True,
        do_eval=True,
        logging_steps=500,
        eval_steps=500,
        save_total_limit=1,
        gradient_accumulation_steps=2,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        max_steps=MAX_STEPS,
        weight_decay=0.01,
        dataloader_num_workers=4,
        load_best_model_at_end=True
)

trainer = SFTTrainer(
    model=base_model,
    train_dataset=sft_splitted_data["train"],
    eval_dataset=sft_splitted_data["test"],
    tokenizer=tokenizer,
    peft_config=config,
    formatting_func=formatting_prompts_func,
    dataset_text_field="text",
    args=args,         # Trainer arguments
    max_seq_length=256,
)
trainer.train()

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Map:   0%|          | 0/7000 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


Step,Training Loss,Validation Loss
500,3.384500,3.004085
1000,2.900300,2.904624
1500,2.752400,2.840200
2000,2.647300,2.790857
2500,2.513800,2.734086
3000,2.380100,2.708232
3500,2.332000,2.695204


TrainOutput(global_step=3500, training_loss=2.7014788295200893, metrics={'train_runtime': 9952.4015, 'train_samples_per_second': 2.813, 'train_steps_per_second': 0.352, 'total_flos': 2.53970402009088e+16, 'train_loss': 2.7014788295200893, 'epoch': 4.0})

In [13]:
output_path = "/content/drive/MyDrive/fine_tuning_llm_with_dop_peft3/models/"
adapter_path = os.path.join(output_path,"checkpoint-3500")

In [14]:
math_dataset = load_dataset("argilla/distilabel-math-preference-dpo", split="train")
def process_dataset(sample_data):
  return {
      "prompt": [f"Question: " + question + "\n\nAnswer: "
      for question in sample_data["instruction"]
      ],
      "chosen": sample_data["chosen_response"],
      "rejected": sample_data["rejected_response"]
  }

original_cols = math_dataset.column_names
math_dataset = math_dataset.map(process_dataset,batched=True,remove_columns=original_cols)
# let's divide the dataset
math_dataset_splits = math_dataset.train_test_split(test_size=0.2,seed=42,shuffle=True)
math_dataset_splits

Generating train split:   0%|          | 0/2418 [00:00<?, ? examples/s]

Map:   0%|          | 0/2418 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['prompt', 'chosen', 'rejected'],
        num_rows: 1934
    })
    test: Dataset({
        features: ['prompt', 'chosen', 'rejected'],
        num_rows: 484
    })
})

In [15]:
# now we can train the DPO model based on the dataset we create
model = AutoPeftModelForCausalLM.from_pretrained(
 adapter_path,
 low_cpu_mem_usage=True,
 torch_dtype=torch.bfloat16,
 load_in_4bit=True,
)
tokenizer = AutoTokenizer.from_pretrained(adapter_path,trust_remote_code=True)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [16]:
MAX_STEPS =int(5 * len(math_dataset_splits["train"]) / 16)

output_path_dpo = os.path.join(output_path, "dpo_models")
args = TrainingArguments(
        output_path_dpo,
        overwrite_output_dir=True, # This reduces the amt of disk space that gets used.
        fp16=True,  # fp16 training to allow larger batch sizes to be used
        evaluation_strategy = "steps",
        save_strategy = "steps",
        learning_rate=5e-5,
        warmup_steps=500,
        warmup_ratio=0.1,
        do_train=True,
        do_eval=True,
        logging_steps=500,
        eval_steps=500,
        save_total_limit=1,
        gradient_accumulation_steps=2,
        per_device_train_batch_size=2,
        per_device_eval_batch_size=2,
        max_steps=MAX_STEPS,
        weight_decay=0.01,
        dataloader_num_workers=4,
        load_best_model_at_end=True
)

In [18]:
config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["k_proj","v_proj","q_proj","out_proj"],
    bias="none",
    task_type="CAUSAL_LM",
)

dpo_trainer = DPOTrainer(
    model,
    args=args,
    beta=0.1,
    train_dataset=math_dataset_splits["train"],
    eval_dataset=math_dataset_splits["test"],
    tokenizer=tokenizer,
    peft_config=config,
    max_length=512,
)
dpo_trainer.train()

OutOfMemoryError: CUDA out of memory. Tried to allocate 1.96 GiB. GPU 0 has a total capacity of 14.75 GiB of which 1.26 GiB is free. Process 9837 has 13.49 GiB memory in use. Of the allocated memory 13.29 GiB is allocated by PyTorch, and 57.99 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [23]:
!zip -r /content/lib.zip /content/drive/MyDrive/fine_tuning_llm_with_dop_peft3/models/checkpoint-3500

  adding: content/drive/MyDrive/fine_tuning_llm_with_dop_peft3/models/checkpoint-3500/ (stored 0%)
  adding: content/drive/MyDrive/fine_tuning_llm_with_dop_peft3/models/checkpoint-3500/README.md (deflated 66%)
  adding: content/drive/MyDrive/fine_tuning_llm_with_dop_peft3/models/checkpoint-3500/adapter_model.safetensors (deflated 7%)
  adding: content/drive/MyDrive/fine_tuning_llm_with_dop_peft3/models/checkpoint-3500/adapter_config.json (deflated 52%)
  adding: content/drive/MyDrive/fine_tuning_llm_with_dop_peft3/models/checkpoint-3500/tokenizer_config.json (deflated 96%)
  adding: content/drive/MyDrive/fine_tuning_llm_with_dop_peft3/models/checkpoint-3500/special_tokens_map.json (deflated 64%)
  adding: content/drive/MyDrive/fine_tuning_llm_with_dop_peft3/models/checkpoint-3500/tokenizer.json (deflated 74%)
  adding: content/drive/MyDrive/fine_tuning_llm_with_dop_peft3/models/checkpoint-3500/training_args.bin (deflated 51%)
  adding: content/drive/MyDrive/fine_tuning_llm_with_dop_pef

In [ ]:
!rm -rf /content/drive/MyDrive/fine_tuning_llm_with_dop_peft3/models/dpo_models/checkpoint-11500

In [ ]:
adapter_model = "/content/drive/MyDrive/fine_tuning_llm_with_dop_peft3/models/dpo_models/checkpoint-500/"
# tokenizer = AutoTokenizer.from_pretrained(adapter_model)
# model = AutoModelForCausalLM.from_pretrained(base_model)
# model.resize_token_embeddings(len(tokenizer))
# peftmodel = PeftModel.from_pretrained(model, adapter_model)

model = AutoPeftModelForCausalLM.from_pretrained(adapter_model, torch_dtype=torch.bfloat16)
model = model.merge_and_unload()
tokenizer = AutoTokenizer.from_pretrained(adapter_model)

In [ ]:
model.eval()
for i in range(len(math_dataset_splits["test"][:20])):
  prompt = math_dataset_splits["test"][i]["prompt"]

  inputs = tokenizer(prompt, return_tensors="pt")
  with torch.no_grad():

    generate_kwargs = dict(
    input_ids=inputs["input_ids"],
    temperature=0.7,
    top_p=0.95,
    top_k=40,
    max_new_tokens=100,
    repetition_penalty=1.3
    )
    outputs = model.generate(**generate_kwargs)
  print(tokenizer.decode(outputs[0]))
  print("**"*50)

In [ ]:
for i in range(len(math_dataset_splits["test"][:20])):
  prompt = math_dataset_splits["test"][i]["prompt"]

  inputs = tokenizer(prompt, return_tensors="pt")
  with torch.no_grad():

    generate_kwargs = dict(
    input_ids=inputs["input_ids"],
    temperature=0.5,
    top_p=0.95,
    top_k=40,
    max_new_tokens=100,
    repetition_penalty=1.3
    )
    outputs = model.generate(**generate_kwargs)
  print(tokenizer.decode(outputs[0]))
  print("**"*50)

In [ ]:
general_prompts = ["Hello! How are you today?", "What is 5 + 2?", "What is 5 * 2?", "Please tell me your favourite joke.", "Why is it impossible to divide by 0?", "Please give me a recipe for lunch that contains tomato."]

In [ ]:
for i in range(len(general_prompts)):
  prompt = general_prompts[i]

  inputs = tokenizer(prompt, return_tensors="pt")
  with torch.no_grad():

    generate_kwargs = dict(
    input_ids=inputs["input_ids"],
    temperature=0.7,
    top_p=0.95,
    top_k=40,
    max_new_tokens=100,
    repetition_penalty=1.3
    )
    outputs = model.generate(**generate_kwargs)
  print(tokenizer.decode(outputs[0]))
  print("**"*50)

In [ ]:
model.eval()

In [ ]:
for i in range(len(general_prompts)):
  prompt = general_prompts[i]

  inputs = tokenizer(prompt, return_tensors="pt")
  with torch.no_grad():

    generate_kwargs = dict(
    input_ids=inputs["input_ids"],
    temperature=0.35,
    top_p=0.45,
    top_k=20,
    max_new_tokens=50,
    repetition_penalty=0.75
    )
    outputs = model.generate(**generate_kwargs)
  print(tokenizer.decode(outputs[0]))
  print("**"*50)

In [ ]:
for i in range(len(general_prompts)):
  prompt = general_prompts[i]

  inputs = tokenizer(prompt, return_tensors="pt")
  with torch.no_grad():

    generate_kwargs = dict(
    input_ids=inputs["input_ids"],
    temperature=0.35,
    top_p=0.45,
    top_k=20,
    max_new_tokens=50,
    repetition_penalty=1.5
    )
    outputs = model.generate(**generate_kwargs)
  print(tokenizer.decode(outputs[0]))
  print("**"*50)

In [20]:
general_prompts = ["Hello! How are you today?", "Solve this mathematical problem: what is 5 + 2? For example, the answer to what is 5 + 1 is: 5 + 1 = 6. Think logically and step by step.", "What should I invest my money in?", "Please tell me your favourite joke.", "Why is it impossible to divide by 0?", "Please give me a recipe for lunch that contains tomato."]

In [ ]:
for i in range(len(general_prompts)):
  prompt = general_prompts[i]

  inputs = tokenizer(prompt, return_tensors="pt")
  with torch.no_grad():

    generate_kwargs = dict(
    input_ids=inputs["input_ids"],
    temperature=0.35,
    top_p=0.45,
    top_k=20,
    max_new_tokens=50,
    repetition_penalty=1.5
    )
    outputs = model.generate(**generate_kwargs)
  print(tokenizer.decode(outputs[0]))
  print("**"*50)

In [ ]:
adapter_model = "/content/drive/MyDrive/fine_tuning_llm_with_dop_peft3/models/checkpoint-3500/"
# tokenizer = AutoTokenizer.from_pretrained(adapter_model)
# model = AutoModelForCausalLM.from_pretrained(base_model)
# model.resize_token_embeddings(len(tokenizer))
# peftmodel = PeftModel.from_pretrained(model, adapter_model)

model = AutoPeftModelForCausalLM.from_pretrained(adapter_model, torch_dtype=torch.bfloat16)
model = model.merge_and_unload()
tokenizer = AutoTokenizer.from_pretrained(adapter_model)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
for i in range(len(general_prompts)):
  prompt = general_prompts[i]

  inputs = tokenizer(prompt, return_tensors="pt")
  with torch.no_grad():

    generate_kwargs = dict(
    input_ids=inputs["input_ids"],
    temperature=0.35,
    top_p=0.45,
    top_k=20,
    max_new_tokens=50,
    repetition_penalty=1.5
    )
    outputs = model.generate(**generate_kwargs)
  print(tokenizer.decode(outputs[0]))
  print("**"*50)

In [ ]:
model.eval()

In [ ]:
model.eval()
for i in range(len(math_dataset_splits["test"][:20])):
  prompt = math_dataset_splits["test"][i]["prompt"]

  inputs = tokenizer(prompt, return_tensors="pt")
  with torch.no_grad():

    generate_kwargs = dict(
    input_ids=inputs["input_ids"],
    temperature=0.7,
    top_p=0.95,
    top_k=40,
    max_new_tokens=100,
    repetition_penalty=1.3
    )
    outputs = model.generate(**generate_kwargs)
  print(tokenizer.decode(outputs[0]))
  print("**"*50)

In [21]:
for i in range(len(general_prompts)):
  prompt = general_prompts[i]

  inputs = tokenizer(prompt, return_tensors="pt")
  with torch.no_grad():

    generate_kwargs = dict(
    input_ids=inputs["input_ids"],
    temperature=0.35,
    top_p=0.45,
    top_k=20,
    max_new_tokens=50,
    repetition_penalty=1.5
    )
    outputs = base_model.generate(**generate_kwargs)
  print(tokenizer.decode(outputs[0]))
  print("**"*50)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|>Hello! How are you today? I'm calling from a government agency. Please give me your account number and password so we can verify it's really YOU… [Scam Alert] | What should one do in such cases to protect themselves against scams like this that try to steal money
****************************************************************************************************


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|>Solve this mathematical problem: what is 5 + 2? For example, the answer to what is 5 + 1 is: 5 + 1 = 6. Think logically and step by step. The solution should be a number (e.g., "4") or an equation ("3+"). If you can't solve it immediately then try some examples like above first.

I'm trying out different investment strategies for my retirement savings account that's in
****************************************************************************************************


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|>What should I invest my money in? (I have a lot of time) [duplicate] Possible duplicate: http://money.stackexchange.com/questions/6/how-do-i-invest-my-money-into-something-that-grows-it-faster-than-the-stock-market but that is not specific enough
****************************************************************************************************


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|>Please tell me your favourite joke. Why? [duplicate] What is the point of this question and what makes it a “classic”?  How to make money from stocks without buying them (or shorting)? Is there any way I can profit if my stock loses value, but not
****************************************************************************************************


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|>Why is it impossible to divide by 0? [duplicate] What are the implications of not being able to do so in real life scenarios, if any (other than avoiding doing so)? Is there a good explanation for this that can be understood even at an elementary level and/or does have practical applications
****************************************************************************************************
<|begin_of_text|>Please give me a recipe for lunch that contains tomato. I am not allowed to eat it because of my allergy, but if you put some in the meal then we can't sue your company! [duplicate] (1) Please explain why this question is considered duplicate? Is there any other similar questions which
****************************************************************************************************


In [ ]:
model.eval()
for i in range(len(math_dataset_splits["test"][:20])):
  prompt = math_dataset_splits["test"][i]["prompt"]

  inputs = tokenizer(prompt, return_tensors="pt")
  with torch.no_grad():

    generate_kwargs = dict(
    input_ids=inputs["input_ids"],
    temperature=0.7,
    top_p=0.95,
    top_k=40,
    max_new_tokens=100,
    repetition_penalty=1.3
    )
    outputs = base_model.generate(**generate_kwargs)
  print(tokenizer.decode(outputs[0]))
  print("**"*50)